In [1]:
import os
import pandas as pd
import uuid
import json
import vertexai
from vertexai import agent_engines
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Callable, Dict, Optional, Union, Any
# Importy z własnych modułów
from config import PROJECT_ID, LOCATION, MEMORY_ENGINE_DISPLAY_NAME, INPUT_FILE_PATH,MAIN_AGENT,CRITIC_MODEL,CODE_MODEL, API_TYPE_GEMINI,API_TYPE_SONNET, ANTHROPIC_API_KEY,basic_config_agent
from agents.state import AgentWorkflowState
from agents.autogen_agents import TriggerAgent,PlannerAgent,CriticAgent
from prompts import AutoGen_Agents_Propmpt
from agents.langgraph_nodes import * 
from agents.autogen_agent_utils import run_autogen_planning_phase
from memory.memory_bank_client import MemoryBankClient
from tools.utils import read_source_code, save_autogen_conversation_log, save_langgraph_execution_log

In [2]:
AGENT_ENGINE_NAME = "" # Zostanie wypełniona po pobraniu lub utworzeniu silnika

# Inicjalizacja głównego klienta Vertex AI
client = vertexai.Client(project=PROJECT_ID, location=LOCATION)

In [3]:
def get_or_create_agent_engine(display_name: str) :
    """
    Pobiera istniejący Agent Engine po nazwie wyświetlanej lub tworzy nowy, jeśli nie istnieje.
    """
    # 1. Pobierz listę wszystkich istniejących silników w projekcie
    all_engines = agent_engines.list()
    
    # 2. Sprawdź, czy któryś z nich ma pasującą nazwę
    for engine in all_engines:
        if engine.display_name == display_name:
            print(f"INFO: Znaleziono i połączono z istniejącym Agent Engine: '{display_name}'")
            return engine
            
    # 3. Jeśli pętla się zakończyła i nic nie znaleziono, stwórz nowy silnik
    print(f"INFO: Nie znaleziono Agent Engine o nazwie '{display_name}'. Tworzenie nowego...")
    try:
        new_engine = agent_engines.create(
            display_name=display_name
        )
        print(f"INFO: Pomyślnie utworzono nowy Agent Engine.")
        return new_engine
    except Exception as e:
        print(f"KRYTYCZNY BŁĄD: Nie można utworzyć Agent Engine. Sprawdź konfigurację i uprawnienia. Błąd: {e}")
        exit()


In [4]:
agent_engine =get_or_create_agent_engine(MEMORY_ENGINE_DISPLAY_NAME)
AGENT_ENGINE_NAME = agent_engine.resource_name
print(AGENT_ENGINE_NAME)


INFO: Znaleziono i połączono z istniejącym Agent Engine: 'memory-gamma-way'
projects/815755318672/locations/us-central1/reasoningEngines/3849548538518175744


In [5]:
# --- Konfiguracja czatu grupowego ---
main_agent_configuration={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(agent_name=MAIN_AGENT, api_type=API_TYPE_GEMINI, location=LOCATION, project_id=PROJECT_ID)}
critic_agent_configuration ={"cache_seed": 42,"seed": 42,"temperature": 0.0,
                        "config_list": basic_config_agent(api_key=ANTHROPIC_API_KEY,agent_name=CRITIC_MODEL, api_type=API_TYPE_SONNET)}
trigger_prompt = str(AutoGen_Agents_Propmpt.Trigger_prompt())
planner_prompt = str(AutoGen_Agents_Propmpt.Planner_prompt())
critic_prompt = str(AutoGen_Agents_Propmpt.Critic_prompt())
#---WYWOŁANIE AGENTÓW
trigger_agent = TriggerAgent(llm_config=main_agent_configuration, prompt=trigger_prompt)
planner_agent = PlannerAgent(llm_config=main_agent_configuration,prompt=planner_prompt)
critic_agent = CriticAgent(llm_config=main_agent_configuration,prompt=critic_prompt)

In [6]:
if __name__ == "__main__":
    os.makedirs("reports", exist_ok=True)
    system_source_code = read_source_code("Agents_beta.ipynb") # Pamiętaj o poprawnej nazwie pliku

    # --- Inicjalizacja Pamięci i Uruchomienia ---
    memory_client = MemoryBankClient(client=client, agent_engine=agent_engine)
    run_id = str(uuid.uuid4())
    
    print("\n--- ODPYTYWANIE PAMIĘCI O INSPIRACJE ---")
    inspiration_prompt = ""
    dataset_signature = ""
    try:
        df_preview = pd.read_csv(INPUT_FILE_PATH, nrows=0)
        dataset_signature = memory_client.create_dataset_signature(df_preview)
        past_memories = memory_client.query_memory(
            query_text="Najlepsze strategie i kluczowe wnioski dotyczące przetwarzania danych",
            scope={"dataset_signature": dataset_signature},
            top_k=3
        )
        if past_memories:
            inspirations = []
            for mem in past_memories:
                if mem.memory_type == MemoryType.SUCCESSFUL_PLAN and 'key_insight' in mem.content:
                    inspirations.append(f"SPRAWDZONY WNIOSEK Z PLANU: {mem.content['key_insight']}")
                elif mem.memory_type == MemoryType.SUCCESSFUL_FIX and 'key_takeaway' in mem.content:
                    inspirations.append(f"NAUCZKA Z NAPRAWIONEGO BŁĘDU: {mem.content['key_takeaway']}")
            if inspirations:
                inspiration_prompt = "--- INSPIRACJE Z POPRZEDNICH URUCHOMIEŃ ---\n" + "\n".join(inspirations)
                print("INFO: Pomyślnie pobrano inspiracje z pamięci.")
        else:
            print("INFO: Nie znaleziono inspiracji w pamięci dla tego typu danych.")
    except Exception as e:
        print(f"OSTRZEŻENIE: Nie udało się pobrać inspiracji z pamięci: {e}")

    # --- Krok 1: Faza planowania AutoGen ---
    final_plan, autogen_log = run_autogen_planning_phase(input_path=INPUT_FILE_PATH, inspiration_prompt=inspiration_prompt,
                                                         trigger_agent=trigger_agent,planner_agent=planner_agent,critic_agent=critic_agent,manager_agent_config=main_agent_configuration)

    # Zapis logu z planowania (zawsze)
    save_autogen_conversation_log(log_content=autogen_log, file_path="reports/autogen_planning_conversation.log")

    # --- Krok 2: Faza wykonania LangGraph ---
    if final_plan:
        print("\n" + "="*80)
        print("### ### FAZA 2: URUCHAMIANIE WYKONANIA PLANU (LangGraph) ### ###")
        print("="*80 + "\n")
        
        workflow = StateGraph(AgentWorkflowState)
        
        # ZMIANA: Dodajemy nowy węzeł commit_memory_node do listy
        nodes = [
            "schema_reader", "code_generator", "architectural_validator", 
            "data_code_executor", "universal_debugger", "apply_code_fix", 
            "human_approval", "package_installer", "reporting_agent", 
            "report_executor", "human_escalation", "sync_report_code",
            "commit_memory" # NOWY WĘZEŁ
        ]
        for name in nodes: workflow.add_node(name, globals()[f"{name}_node"])

        # --- Definicja Krawędzi Grafu ---
        workflow.set_entry_point("schema_reader")
        workflow.add_edge("schema_reader", "code_generator")
        workflow.add_edge("code_generator", "architectural_validator")

        # Funkcja routująca, której możemy używać wielokrotnie
        def should_continue_or_debug(state: AgentWorkflowState) -> str:
            """Sprawdza, czy w stanie jest błąd i decyduje o dalszej ścieżce."""
            if state.get("error_message"):
                if state.get("correction_attempts", 0) >= MAX_CORRECTION_ATTEMPTS:
                    return "request_human_help"
                return "call_debugger"
            # Jeśli nie ma błędu, kontynuuj normalną ścieżkę
            return "continue"

        # 1. KRAWĘDŹ WARUNKOWA po walidatorze architektury (KLUCZOWA ZMIANA)
        workflow.add_conditional_edges(
            "architectural_validator",
            should_continue_or_debug,
            {
                "call_debugger": "universal_debugger",
                "request_human_help": "human_escalation",
                "continue": "data_code_executor" # Przejdź dalej tylko jeśli jest OK
            }
        )

        # 2. KRAWĘDŹ WARUNKOWA po wykonaniu kodu danych
        workflow.add_conditional_edges(
            "data_code_executor",
            should_continue_or_debug,
            {
                "call_debugger": "universal_debugger",
                "request_human_help": "human_escalation",
                "continue": "commit_memory" # Jeśli sukces, idź do zapisu w pamięci, a NIE do END
            }
        )

        # Ścieżka sukcesu i pozostałe krawędzie
        workflow.add_edge("commit_memory", "reporting_agent")
        workflow.add_edge("reporting_agent", "report_executor")

        # Krawędź warunkowa po wykonaniu raportu
        workflow.add_conditional_edges(
            "report_executor",
            should_continue_or_debug,
            {
                "call_debugger": "universal_debugger",
                "request_human_help": "human_escalation",
                "continue": END # Dopiero tutaj kończymy pracę po sukcesie
            }
        )

        # Ścieżki naprawcze i eskalacji (bez zmian)
        workflow.add_edge("human_escalation", END)
        workflow.add_edge("package_installer", "data_code_executor") # Wracamy do wykonania po instalacji

        def route_after_fix(state):
            failing_node = state.get("failing_node")
            if failing_node == "report_executor":
                return "sync_report_code"
            # Po każdej innej naprawie wracamy do walidacji architektonicznej
            return "architectural_validator"

        workflow.add_edge("sync_report_code", "report_executor")
        workflow.add_conditional_edges("apply_code_fix", route_after_fix)

        def route_from_debugger(state):
            if state.get("tool_choice") == "propose_code_fix":
                return "apply_code_fix"
            if state.get("tool_choice") == "request_package_installation":
                return "human_approval"
            return "human_escalation"

        workflow.add_conditional_edges("universal_debugger", route_from_debugger)
        workflow.add_conditional_edges("human_approval", lambda s: s.get("user_approval_status"), {
            "APPROVED": "package_installer",
            "REJECTED": "universal_debugger"
        })

        
        
        app_config ={"MAIN_AGENT" : MAIN_AGENT, "CODE_MODEL": CODE_MODEL, "CRITIC_MODEL":CRITIC_MODEL}
        
        
        app = workflow.compile()
        
        initial_state = {
            "config":app_config,
            "plan": final_plan, 
            "input_path": INPUT_FILE_PATH,
            "output_path": "reports/processed_data.csv",
            "report_output_path": "reports/transformation_report.html",
            "correction_attempts": 0, 
            "source_code": system_source_code,
            "autogen_log": autogen_log,
            "memory_client": memory_client,
            "run_id": run_id,
            "dataset_signature": dataset_signature,
            "pending_fix_session": None # ZMIANA: Dodanie nowego pola do stanu początkowego
        }
        
        # --- Uruchomienie grafu z przechwytywaniem logów ---
        langgraph_log = ""
        final_run_state = initial_state.copy()
        
        for event in app.stream(initial_state, {"recursion_limit": 50}):
            for node_name, state_update in event.items():
                if "__end__" not in node_name:
                    print(f"--- Krok: '{node_name}' ---")
                    if state_update: # Zabezpieczenie przed błędem 'NoneType'
                        printable_update = state_update.copy()
                        for key in ["generated_code", "corrected_code", "generated_report_code", "error_context_code"]:
                            if key in printable_update and printable_update[key]:
                                print(f"--- {key.upper()} ---")
                                print(printable_update[key])
                                print("-" * (len(key) + 8))
                                del printable_update[key]
                        if printable_update:
                            print(json.dumps(printable_update, indent=2, default=str))
                        
                        log_line = f"--- Krok: '{node_name}' ---\n{json.dumps(state_update, indent=2, default=str)}\n"
                        langgraph_log += log_line
                        final_run_state.update(state_update)
                    else:
                        print("  [INFO] Węzeł zakończył pracę bez aktualizacji stanu.")
                    print("-" * 20 + "\n")

        # Zapis logu z wykonania (po zakończeniu pętli)
        save_langgraph_execution_log(log_content=langgraph_log, file_path="reports/langgraph_execution.log")

        # Uruchomienie audytora
        final_run_state['langgraph_log'] = langgraph_log
        meta_auditor_node(final_run_state)

        print("\n\n--- ZAKOŃCZONO PRACĘ GRAFU I AUDYT ---")
    else:
        print("Proces zakończony. Brak planu do wykonania.")

INFO: MemoryBankClient gotowy do pracy z silnikiem: projects/815755318672/locations/us-central1/reasoningEngines/3849548538518175744

--- ODPYTYWANIE PAMIĘCI O INSPIRACJE ---
INFO: Odpytuję pamięć semantycznie z zapytaniem 'Najlepsze strategie i kluczowe wnioski dotyczące przetwarzania danych' w zakresie {'dataset_signature': 'ae1568fe7dae11d4bacd0c21ed718503'}


/home/jupyter/olga_zydziak/version_beta/multiagent_system/memory/memory_bank_client.py:95: ExperimentalWarning: The Vertex SDK GenAI agent engines module is experimental, and may change in future versions.
  memories_iterator = self.client.agent_engines.retrieve_memories(


INFO: Znaleziono i poprawnie przetworzono 0 pasujących wspomnień.
INFO: Nie znaleziono inspiracji w pamięci dla tego typu danych.

### ### FAZA 1: URUCHAMIANIE PLANOWANIA STRATEGICZNEGO (AutoGen) ### ###

UserProxy (to chat_manager):

Oto podgląd danych:

Kolumny:
['Transaction_ID', 'User_ID', 'Transaction_Amount', 'Transaction_Type', 'Timestamp', 'Account_Balance', 'Device_Type', 'Location', 'Merchant_Category', 'IP_Address_Flag', 'Previous_Fraudulent_Activity', 'Daily_Transaction_Count', 'Avg_Transaction_Amount_7d', 'Failed_Transaction_Count_7d', 'Card_Type', 'Card_Age', 'Transaction_Distance', 'Authentication_Method', 'Risk_Score', 'Is_Weekend', 'Fraud_Label']

Pierwsze 5 wierszy:
  Transaction_ID    User_ID  Transaction_Amount Transaction_Type            Timestamp  Account_Balance Device_Type  Location Merchant_Category  IP_Address_Flag  Previous_Fraudulent_Activity  Daily_Transaction_Count  Avg_Transaction_Amount_7d  Failed_Transaction_Count_7d   Card_Type  Card_Age  Transaction_D